# Elasticsearch With Haystack

We will be communicating with our Elasticsearch document store via Haystack. First, we need to install Haystack using pip:

On Windows:

```
pip install farm-haystack -f https://download.pytorch.org/whl/torch_stable.html
```

Anything else:

```
pip install farm-haystack
```

We will start by indexing the SQuAD dev data. So let's load that into our notebook first.



In [1]:
# !pip install farm-haystack

In [2]:
# !pip install farm-haystack[elasticsearch]

In [3]:
import json

with open('../../data/squad/dev.json', 'r') as f:
    squad = json.load(f)

In [4]:
squad[:2]

[{'question': 'In what country is Normandy located?',
  'answer': 'France',
  'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.'},
 {'question': 'When were the Normans in Normandy?',
  'answer': '10th and 11th centuries',
  'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: No

Next, we initialize a connection between Haystack and our local Elasticsearch instance like so:

In [5]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host='localhost', username='', password='', index='squad_docs')

Great, we've established our connection, now let's try querying our Elasticsearch instance. We will do this through the `requests` library.

In [6]:
import requests

Let's check our cluster *health* (eg the general status of our Elasticsearch instance). We do this by sending a **GET** request to the `_cluster/health` endpoint.

In [7]:
res = requests.get('http://localhost:9200/_cluster/health')

res.json()

{'cluster_name': 'elasticsearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 3,
 'active_shards': 3,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 60.0}

Okay we can see that the cluster is definitely running. The cluster status is *yellow*, ideally we want to aim for *green* but the reason we see yellow here is because not all replica shards have been allocated to nodes. The details of this don't really matter, but it essentially just means that we don't have a full set of backup (*replica*) data shards - which is only a problem if our *primary* data sources get corrupted/lost. That is beyond the scope of what we are doing here however.

## Adding Data

Right now our Elasticsearch instance contains a single, empty index called *'squad_docs'*. We need to populate this with our `squad` data. We populate our index through the `document_store.write_documents(<input_data>)` method, where our *\<input_data\>* must be a list of dictionaries in the format:

```json
{
    'content': '<document text here>',
    'meta': {
        'other': '<other info here>'
    }
}
```

We **must** include the `'content'` key. The *content* must contain the text from each sample, which in our case is a *context* string. The `'meta'` data is optional, but is usually used to contain anything else that might be relevant, so for example we might want to include the *group* that the context came from (eg 'Beyonce', or 'Matter').
https://github.com/deepset-ai/haystack/issues/1668

In [8]:
squad_docs = []

for sample in squad:
    squad_docs.append({
        'content': sample['context']
    })

In [9]:
squad_docs[:2]

[{'content': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.'},
 {'content': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norsem

Then we add our data to the index like this:

In [10]:
document_store.write_documents(squad_docs)

### Retrieving data ###

When we're retrieving data from Elasticsearch we will be retrieving documents using either the TF-IDF, or BM25 algorithms.

**TF-IDF** is a common *relevance* scoring algorithm, the built is calculated using:

* **TF**, the volume of words in the query (question) that appear in the document.

* **IDF**, the inverse of the fraction of documents that contain the same word (eg common words like *'the'* don't score well, whereas *'Beyonce'* would).

We integrate TD-IDF using:

In [11]:
from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store)
retriever.document_counts

{'squad_docs': 1204}

We can see here that when building our retriever, it identified a total of *16209* 'candidate paragraphs'. These are all of the contexts from our `squad` data:

In [12]:
len(squad)

10279

For now, we can return data from Elasticsearch, using the **TF-IDF** algorithm, with the `retrieve` method.

In [13]:
query = "Physics is a very abstract subject"

retriever.retrieve(query)

[<Document: {'content': 'Even though some proofs of complexity-theoretic theorems regularly assume some concrete choice of input encoding, one tries to keep the discussion abstract enough to be independent of the choice of encoding. This can be achieved by ensuring that different representations can be transformed into each other efficiently.', 'content_type': 'text', 'score': None, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': 'fdafa6182e072da1f6005c49f858c175'}>,
 <Document: {'content': 'With modern insights into quantum mechanics and technology that can accelerate particles close to the speed of light, particle physics has devised a Standard Model to describe forces between particles smaller than atoms. The Standard Model predicts that exchanged particles called gauge bosons are the fundamental means by which forces are emitted and absorbed. Only four main interactions are known: in order of decreasing strength, they are: strong, electromagnetic, weak, and gravit

This query returns a huge number of duplicates. The reason we have these is because our data contained duplicates of the same context because each context could be tied to several different questions. So now, we need to restart by first deleting everything inside our *squad_docs* index. Then re-indexing our deduplicated data.

We can delete every document in our index by sending a **POST** request to the `<index_name>/_delete_by_query` endpoint:

In [14]:
res = requests.post('http://localhost:9200/squad_docs/_delete_by_query',
                    json={
                        'query': {
                            'match_all': {}
                        }
                    })

res.json()

{'took': 55,
 'timed_out': False,
 'total': 1204,
 'deleted': 1204,
 'batches': 2,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

Our response shows `'deleted': 16209`, which means all *16209* documents have been deleted from our *squad_docs* index. We can confirm this by calling the `<index_name>/_count` endpoint too:

In [22]:
res = requests.get('http://localhost:9200/squad_docs/_count')

res.json()

{'count': 1204,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

Now that we've cleared the index, it's time to remove duplicates from our SQuAD contexts and re-index them.

In [16]:
# create list of contexts (we cannot do this using current dictionary format)
contexts = [sample['context'] for sample in squad]

# convert to set to remove duplicates, then back to list
contexts = list(set(contexts))

# convert back to dictionary format we need
squad_docs = [{'content': sample} for sample in contexts]

Finally, we can re-index our Elasticsearch as we did before.

In [17]:
document_store.write_documents(squad_docs)

Because we have changed the contents of our index, we initialize our retriever once more.

In [18]:
retriever = TfidfRetriever(document_store)

In [19]:
retriever.document_counts

{'squad_docs': 1204}

And this time we see that our retriever found *1204* documents (much less than the *16209* we found before). Now it's time to query our data again!

In [20]:
retriever.retrieve(query)

[<Document: {'content': 'Even though some proofs of complexity-theoretic theorems regularly assume some concrete choice of input encoding, one tries to keep the discussion abstract enough to be independent of the choice of encoding. This can be achieved by ensuring that different representations can be transformed into each other efficiently.', 'content_type': 'text', 'score': None, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': 'fdafa6182e072da1f6005c49f858c175'}>,
 <Document: {'content': 'With modern insights into quantum mechanics and technology that can accelerate particles close to the speed of light, particle physics has devised a Standard Model to describe forces between particles smaller than atoms. The Standard Model predicts that exchanged particles called gauge bosons are the fundamental means by which forces are emitted and absorbed. Only four main interactions are known: in order of decreasing strength, they are: strong, electromagnetic, weak, and gravit

Now we're returning a set of relevant documents, without duplicates.

Finally, let's return back to the other *sparse retriever* that we can use with Elasticsearch. We already used **TF-IDF**, by switching `TfidfRetriever` for `ElasticsearchRetriever` we can switch to the **BM25** algorithm, which is an *improved* version of **TF-IDF** and is recommended by Haystack.

So, let's initialize that and make another query with it.

In [21]:
# import BM25 retriever
# from haystack.retriever.sparse import ElasticsearchRetriever  # old
from haystack.nodes import BM25Retriever


# intialize
retriever = BM25Retriever(document_store)

# and query
retriever.retrieve(query)

[<Document: {'content': 'Even though some proofs of complexity-theoretic theorems regularly assume some concrete choice of input encoding, one tries to keep the discussion abstract enough to be independent of the choice of encoding. This can be achieved by ensuring that different representations can be transformed into each other efficiently.', 'content_type': 'text', 'score': 0.7327450739934891, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': 'fdafa6182e072da1f6005c49f858c175'}>,
 <Document: {'content': 'A computational problem can be viewed as an infinite collection of instances together with a solution for every instance. The input string for a computational problem is referred to as a problem instance, and should not be confused with the problem itself. In computational complexity theory, a problem refers to the abstract question to be solved. In contrast, an instance of this problem is a rather concrete utterance, which can serve as the input for a decision probl

Okay great, this is a pretty big notebook but it covers everything we need to know to get started with Haystack + Elastic (and a little more).